In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from tree import AFTSurvivalTree
from forest import AFTForest
from dataset import SupportDataset, SyntheticDataset, VeteranLungDataset
import time

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('../data/veterans_lung_cancer.csv')
df.describe()

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
count,137.000000,137.0,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000,137.000000
mean,121.627737,inf,58.306569,58.569343,8.773723,0.197080,0.197080,0.350365,0.255474,0.708029,0.291971,0.503650,0.496350
std,157.816722,NaN,10.541628,20.039592,10.612141,0.399253,0.399253,0.478835,0.437728,0.456337,0.456337,0.501821,0.501821
min,1.000000,1.0,34.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,25.0,51.000000,40.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,80.0,62.000000,60.000000,5.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,144.000000,177.0,66.000000,75.000000,11.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,999.000000,inf,81.000000,99.000000,87.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df

,Survival_label_lower_bound,Survival_label_upper_bound,Age_in_years,Karnofsky_score,Months_from_Diagnosis,Celltype=adeno,Celltype=large,Celltype=smallcell,Celltype=squamous,Prior_therapy=no,Prior_therapy=yes,Treatment=standard,Treatment=test
0,72.0,72.0,69.0,60.0,7.0,0,0,0,1,1,0,1,0
1,411.0,411.0,64.0,70.0,5.0,0,0,0,1,0,1,1,0
2,228.0,228.0,38.0,60.0,3.0,0,0,0,1,1,0,1,0
3,126.0,126.0,63.0,60.0,9.0,0,0,0,1,0,1,1,0
4,118.0,118.0,65.0,70.0,11.0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,133.0,133.0,65.0,75.0,1.0,0,1,0,0,1,0,0,1
133,111.0,111.0,64.0,60.0,5.0,0,1,0,0,1,0,0,1
134,231.0,231.0,67.0,70.0,18.0,0,1,0,0,0,1,0,1
135,378.0,378.0,65.0,80.0,4.0,0,1,0,0,1,0,0,1


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, genextreme, gaussian_kde, ecdf

In [5]:
veteran = VeteranLungDataset(df)
veteran.get_data()

array([[69., 60.,  7., ...,  0.,  1.,  0.],
       [64., 70.,  5., ...,  1.,  1.,  0.],
       [38., 60.,  3., ...,  0.,  1.,  0.],
       ...,
       [67., 70., 18., ...,  1.,  0.,  1.],
       [65., 80.,  4., ...,  0.,  0.,  1.],
       [37., 30.,  3., ...,  0.,  0.,  1.]])

In [6]:
veteran.get_label()

array([(0,  72.), (0, 411.), (0, 228.), (0, 126.), (0, 118.), (0,  10.),
       (0,  82.), (0, 110.), (0, 314.), (1, 100.), (0,  42.), (0,   8.),
       (0, 144.), (1,  25.), (0,  11.), (0,  30.), (0, 384.), (0,   4.),
       (0,  54.), (0,  13.), (1, 123.), (1,  97.), (0, 153.), (0,  59.),
       (0, 117.), (0,  16.), (0, 151.), (0,  22.), (0,  56.), (0,  21.),
       (0,  18.), (0, 139.), (0,  20.), (0,  31.), (0,  52.), (0, 287.),
       (0,  18.), (0,  51.), (0, 122.), (0,  27.), (0,  54.), (0,   7.),
       (0,  63.), (0, 392.), (0,  10.), (0,   8.), (0,  92.), (0,  35.),
       (0, 117.), (0, 132.), (0,  12.), (0, 162.), (0,   3.), (0,  95.),
       (0, 177.), (0, 162.), (0, 216.), (0, 553.), (0, 278.), (0,  12.),
       (0, 260.), (0, 200.), (0, 156.), (1, 182.), (0, 143.), (0, 105.),
       (0, 103.), (0, 250.), (0, 100.), (0, 999.), (0, 112.), (1,  87.),
       (1, 231.), (0, 242.), (0, 991.), (0, 111.), (0,   1.), (0, 587.),
       (0, 389.), (0,  33.), (0,  25.), (0, 357.), 

In [7]:
veteran.get_xgboost_label()

,Survival_label_lower_bound,Survival_label_upper_bound
0,72.0,72.0
1,411.0,411.0
2,228.0,228.0
3,126.0,126.0
4,118.0,118.0
...,...,...
132,133.0,133.0
133,111.0,111.0
134,231.0,231.0
135,378.0,378.0


In [8]:
X_train, X_test, y_train, y_test = veteran.get_train_test()

##### Init All Methods

AFT-Forest

In [9]:
aft_surv_tree = AFTSurvivalTree(function="norm")
aft_forest = AFTForest(n_trees=10, random_params=True)

In [10]:
aft_surv_tree.fit(X_train, y_train)

In [11]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/m

In [12]:
aft_surv_tree.predict(X_test)

[125.5,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 70.2,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 127.28915662650603,
 104.1,
 127.28915662650603]

In [13]:
aft_forest.predict(X_test)

[120.14440091915526,
 119.6338374144741,
 92.78717869693304,
 118.8788374144741,
 90.26717869693303,
 127.14606710158435,
 138.32750688226125,
 121.98410057236883,
 122.1788374144741,
 121.39883741447412,
 110.45606710158434,
 112.57551203026637,
 140.62285571947055,
 132.18841127502608,
 128.47685328748997,
 121.89952238613716,
 121.98410057236883,
 132.18841127502608,
 128.2821164453847,
 132.94341127502608,
 108.28118905280385,
 121.98410057236883,
 140.62285571947055,
 119.38940091915526,
 90.26717869693303,
 130.75861799337235,
 108.28118905280385,
 127.69685328748997]

In [14]:
aft_surv_tree._score(X_test, y_test)

0.5206043956043956

In [15]:
aft_surv_tree._visualize()

In [16]:
aft_surv_tree._brier(X_test, y_test)

0.10341047841047839

In [17]:
aft_surv_tree._mae(X_test, y_test)

99.91157487091223

In [18]:
aft_surv_tree._auc(X_test, y_test)

(array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.49487179, 0.5       , 0.50274725,
        0.5085034 , 0.46071429, 0.47152194, 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5 

In [19]:
aft_forest._score(X_test, y_test)

0.6648351648351648

In [20]:
aft_forest._brier(X_test, y_test)

0.09980297480297479

In [21]:
aft_forest._auc(X_test, y_test)

(array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.42857143,
        0.47534014, 0.48392857, 0.42086835, 0.3050887 , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5 

In [22]:
aft_surv_tree_weibull = AFTSurvivalTree(function="weibull", is_bootstrap=False, is_custom_dist=True)


In [23]:
aft_surv_tree_weibull.fit(X_train, y_train)

In [24]:

aft_surv_tree_weibull._score(X_test, y_test)

0.5

In [25]:
aft_surv_tree_weibull.save("tree_weibull.json")

In [26]:
aft_surv_tree_load_test = AFTSurvivalTree()
aft_surv_tree_load_test = aft_surv_tree_load_test.load("tree_weibull.json")

In [27]:
aft_surv_tree_load_test._score(X_test, y_test)

0.5

In [28]:
aft_surv_tree_logistic = AFTSurvivalTree(function="logistic", is_bootstrap=False, is_custom_dist=True)

In [29]:
aft_surv_tree_logistic.fit(X_train, y_train)

In [30]:
aft_surv_tree_logistic._score(X_test, y_test)

0.5

In [31]:
aft_surv_tree_gmm = AFTSurvivalTree(function="gmm", is_bootstrap=False, is_custom_dist=True)

In [32]:
aft_surv_tree_gmm.fit(X_train, y_train)

In [33]:
aft_surv_tree_gmm._score(X_test, y_test)

0.570054945054945

In [34]:
aft_surv_tree_normal = AFTSurvivalTree(function="norm", is_bootstrap=False, is_custom_dist=True)

In [35]:
aft_surv_tree_normal.fit(X_train, y_train)


In [36]:
aft_surv_tree_normal._score(X_test, y_test)

0.5

In [37]:
aft_surv_tree_extreme = AFTSurvivalTree(function="extreme", is_bootstrap=True, is_custom_dist=True)

In [38]:
aft_surv_tree_extreme.fit(X_train, y_train)

In [39]:
aft_surv_tree_extreme._score(X_test, y_test)

0.5370879120879121

In [40]:
aft_surv_tree_extreme._visualize()

In [41]:
aft_surv_tree_extreme.save("extreme.json")

In [ ]:
kwargs = {
    "function": "gmm", 
    "is_bootstrap": False, 
    "is_custom_dist": True,
    "n_components": 13
}
aft_forest = AFTForest(n_trees=5, random_params=False, **kwargs)

In [43]:
aft_forest.fit(X_train, y_train)

/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/rizkiduwinanto/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (25,) + inhomogeneous part.

In [ ]:
aft_forest._score(X_test, y_test)

In [ ]:
aft_forest.save("forest.json")

In [ ]:
# aft_forest_load_test._score(X_test, y_test)

In [ ]:
# aft_forest_load_test.save("forest_load.json")

XGBoost-AFT

In [ ]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit


In [ ]:
X_train, X_test, y_train, y_test = veteran.get_train_test_xgboost()

X_train, X_test, y_train, y_test


In [ ]:

dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])
dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])


In [ ]:


params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.20,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=10000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=50)


In [ ]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

In [ ]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [ ]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

# Support2

In [ ]:
df = pd.read_csv('../data/support2.csv')

In [ ]:
data = SupportDataset(df)

In [ ]:

d = data.get_data()
d

In [ ]:
X_train, X_test, y_train, y_test = data.get_train_test_xgboost()

In [ ]:
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])

In [ ]:

dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])

In [ ]:
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.2,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=10)

In [ ]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

In [ ]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [ ]:
y_test.head()

In [ ]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index